<h2 align='center'>Codebasics ML Course: ML Flow Dagshub Tutorial</h2>

In [2]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### Track Experiments

In [6]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    )
]

In [7]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [8]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

# dagshub setup
import dagshub
dagshub.init(repo_owner='auditoria.SGBA1', repo_name='SGBA1-smartgrids', mlflow=True)

with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as auditoria.SGBA1

Initialized MLflow to track repo "auditoria.SGBA1/SGBA1-smartgrids"

Repository auditoria.SGBA1/SGBA1-smartgrids initialized!

🏃 View run aged-fox-509 at: https://dagshub.com/auditoria.SGBA1/SGBA1-smartgrids.mlflow/#/experiments/0/runs/6529fcfe738c4f318646a24bd8a1abeb
🧪 View experiment at: https://dagshub.com/auditoria.SGBA1/SGBA1-smartgrids.mlflow/#/experiments/0


In [9]:
# Initialize MLflow
mlflow.set_experiment("Prueba 2 - Mlflow y DagsHub desde GitHub")
# mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/02/05 19:32:22 INFO mlflow.tracking.fluent: Experiment with name 'Prueba 2 - Mlflow y DagsHub desde GitHub' does not exist. Creating a new experiment.
2025/02/05 19:32:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: https://dagshub.com/auditoria.SGBA1/SGBA1-smartgrids.mlflow/#/experiments/2/runs/695299faf6c94c25b3f7b4f5edbd8280
🧪 View experiment at: https://dagshub.com/auditoria.SGBA1/SGBA1-smartgrids.mlflow/#/experiments/2


2025/02/05 19:32:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: https://dagshub.com/auditoria.SGBA1/SGBA1-smartgrids.mlflow/#/experiments/2/runs/0e8a7d7216ce4a0b9def476be31fd86b
🧪 View experiment at: https://dagshub.com/auditoria.SGBA1/SGBA1-smartgrids.mlflow/#/experiments/2


2025/02/05 19:32:50 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\anaig\AppData\Local\Temp\tmpa2ebe51y\model, flavor: xgboost). Fall back to return ['xgboost==2.1.3']. Set logging level to DEBUG to see the full traceback. 
2025/02/05 19:32:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: https://dagshub.com/auditoria.SGBA1/SGBA1-smartgrids.mlflow/#/experiments/2/runs/f6d8a998a6cb4df58466bd739c5f75c8
🧪 View experiment at: https://dagshub.com/auditoria.SGBA1/SGBA1-smartgrids.mlflow/#/experiments/2
